In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!nvidia-smi

Wed Dec 21 12:33:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from transformers import BertForMaskedLM, BertTokenizer,Trainer,DataCollatorForLanguageModeling


tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
model = BertForMaskedLM.from_pretrained('beomi/kcbert-base')

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
import os
!pwd
os.chdir('/content/drive/MyDrive/Colab Notebooks/Bert_For_Fine_Tuning')
!pwd

/content
/content/drive/MyDrive/Colab Notebooks/Bert_For_Fine_Tuning


In [7]:
from datasets import load_dataset

train = load_dataset('csv',data_files='data/book_train_128.csv')
validation = load_dataset('csv',data_files='data/book_validation_128.csv')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
def tokenize_function(examples):
    return tokenizer(examples['sen'], max_length=128, padding=True, truncation=True)

train_data_set = train['train'].map(tokenize_function,batch_size=True)
validation_data_set = validation['train'].map(tokenize_function,batch_size=True)

In [9]:
from transformers import TrainingArguments, TrainerCallback

### Training Arguments
device = 'cuda'

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_eval_batch_size=64,
    per_device_train_batch_size=64,
    logging_steps=100,
    num_train_epochs=2,
    evaluation_strategy='epoch'
)

data_collator_BERT = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15, return_tensors="pt"
)


class myCallback(TrainerCallback):
    """
    A bare :class:`~transformers.TrainerCallback` that just prints the logs.
    """

    def on_step_begin(self, args, state, control, logs=None, **kwargs):

        # 함수 이름.. 언제 시작할지
        # log는 설정할 때마다
        # arg,state,control은 참고할 수 있는 attribute의 경우임.
        # 근데 내가 필요한건 input
        if state.global_step % args.logging_steps == 0:
            print("")
            print(
                f"{int(state.epoch)}번째 epoch 진행 중 ------- {state.global_step}번째 step 결과"
            )


class customtrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    ############# 내용 추가
    def step_check(self):
        # state는 현 상태를 담는 attribute임.
        return self.state.global_step

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        ############# 내용 추가
        if self.step_check() % self.args.logging_steps == 0:
            # step_check = 현 step 파악
            # args.logging_steps = argument에서 지정한 step 불러오가

            # batch 중 0 번째 위치한 문장 선택
            num = 1
            input_id = inputs.input_ids[num].reshape(-1).data.tolist()
            output_id = outputs.logits[num].argmax(dim=-1).reshape(-1).data.tolist()
            attention_mask = inputs.attention_mask[num]

            # mask가 위치한 idx 추출하기
            mask_idx = (inputs.input_ids[num] == 4).nonzero().data.reshape(-1).tolist()

            # padding 제거
            input_id_without_pad = [
                input_id[i] for i in range(len(input_id)) if attention_mask[i]
            ]
            output_id_without_pad = [
                output_id[i] for i in range(len(output_id)) if attention_mask[i]
            ]

            # id to token
            # [1:-1] [CLS,SEP] 제거
            inputs_tokens = self.tokenizer.convert_ids_to_tokens(input_id_without_pad)[
                1:-1
            ]
            outputs_tokens = self.tokenizer.convert_ids_to_tokens(
                output_id_without_pad
            )[1:-1]

            # output mask 부분 표시하기
            for i in mask_idx:
                # [CLS,SEP 위치 조정]
                outputs_tokens[i - 1] = "[" + outputs_tokens[i - 1] + "]"

            inputs_sen = self.tokenizer.convert_tokens_to_string(inputs_tokens)
            outputs_sen = self.tokenizer.convert_tokens_to_string(outputs_tokens)

            print(f"input 문장 : {''.join(inputs_sen)}")
            print(f"output 문장 : {''.join(outputs_sen)}")

        return (loss, outputs) if return_outputs else loss


trainer = customtrainer(
    model=model.to(device),
    train_dataset=train_data_set,
    eval_dataset=validation_data_set,
    data_collator=data_collator_BERT,
    args=training_args,
    tokenizer=tokenizer,
    callbacks=[myCallback],
)

trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: Unnamed: 0, sen. If Unnamed: 0, sen are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 175900
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 5498
  Number of trainable parameters = 108950064



0번째 epoch 진행 중 ------- 0번째 step 결과
input 문장 : 장 수학 기호 수식에 [MASK] 쓰이는 그리스 알 [MASK]벳을 읽고 [MASK] 법을 배 [MASK]니다
output 문장 : 장르 기호 수식에 [##서] 쓰이는군요 알 [##파]벳을 읽고 [배우는] 법을 배 [##웁]니다


Step,Training Loss
100,3.163900
200,2.946900
300,2.840500
400,2.829800
500,2.730200
600,2.740500
700,2.701600
800,2.638800
900,2.680800
1000,2.598500



0번째 epoch 진행 중 ------- 100번째 step 결과
input 문장 : 물론 [MASK] [MASK] [MASK] 이루어졌고 페 [MASK] [MASK] 어떻게 구성되었 [MASK] 보여주는 것들도 부분부분 있어서 [MASK]지
output 문장 : 물론 [페] [##적인] [어떻게] 이루어졌고 페 [##이지] [##이] 어떻게 구성되었 [##는지] 보여주는 것들도 부분부분 있어서 [좋았]지

0번째 epoch 진행 중 ------- 200번째 step 결과
input 문장 : 이를테 [MASK] 신체화 떙 embody 라는 동사와 신체화 embodiment 라는 [MASK]사는 인간이 무언 [MASK] 우리의 [MASK]화 한다는 [MASK]
output 문장 : 이를테 [##면] 신체화와 embody 라는 동사와 신체화 embodiment 라는 [동]사는 인간이 무언 [##가] 우리의 [시각]화 한다는 [것이다]

0번째 epoch 진행 중 ------- 300번째 step 결과
input 문장 : 윈도우 을 기반으로 기본적으로 사용하는 파 [MASK]과 폴더 그리고 [MASK] 사용 방법에 대해 알려줍니다
output 문장 : 윈도우 를 기반으로 기본적으로 사용하는 파 [##일]과 폴더 그리고 [그] 사용 방법에 대해 알려줍니다

0번째 epoch 진행 중 ------- 400번째 step 결과
input 문장 : 핵심만 [MASK] [MASK] 골라 수험생들의 시간을 아껴드리 [MASK]
output 문장 : 핵심만 [핵심] [##하게] 골라 수험생들의 시간을 아껴드리 [##겠습니다]


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-500/special_tokens_map.json


...


1번째 epoch 진행 중 ------- 4500번째 step 결과
input 문장 : 아마존과 달리 칭찬 일색으로만 도배하는 우리나라의 책 Re [MASK]iew는 도대체 어떻게 믿을 수 있나 모르겠다 .
output 문장 : 아마존과 달리 칭찬 일색으로만 도배하는 이 책 Re [##v]iew는 도대체 어떻게 믿을 수 있나 모르겠다 .

1번째 epoch 진행 중 ------- 4600번째 step 결과
input 문장 : 책에서는 [MASK] 딥 [MASK]닝은 머신 러닝의 많은 기법 중 하나이다 .
output 문장 : 책에서는 [말하는] 딥 [##러]닝은 머신 러닝의 많은 기법 중 하나이다 .

1번째 epoch 진행 중 ------- 4700번째 step 결과
input 문장 : 분할 정복 동적 프로 [MASK]밍 탐욕 [MASK] [MASK] 등 [MASK] [MASK] 문제 해결 전략 [MASK] 학습합니다
output 문장 : 분할 정복 동적 프로 [##그래]밍과 [##의] [방법] 등 [동] [##적인] 문제 해결 전략 [##을] 학습합니다

1번째 epoch 진행 중 ------- 4800번째 step 결과
input 문장 : 본 [MASK]개정 판은 [MASK]킷런 learn 업데이 [MASK]에 따라 전반적으로 내용을 갱신한 원서 쇄를 기반 [MASK] 합니다
output 문장 : 본 [문]개정 판은 [사이]킷런 learn 업데이 [##트]에 따라 전반적으로 내용을 갱신한 원서 코드를 기반 [##으로] 합니다

1번째 epoch 진행 중 ------- 4900번째 step 결과
input 문장 : 엔 [MASK]어의 저작은 대 [MASK] 당대의 첨단 결과상만 집중 조명하기에 현황을 [MASK]기에는 좋아도 응용 [MASK] 유효기간에 한계가 뚜렷한 지식이라는 약점이 있습니다 .
output 문장 : 엔 [##지니]어의 저작은 대 [##략] 당대의 첨단 결과상만 집중 조명하기에 현황을 [따라가]기에는 좋아도 응용 [##은] 유효기간에 한

Saving model checkpoint to test_trainer/checkpoint-5000
Configuration saved in test_trainer/checkpoint-5000/config.json
Model weights saved in test_trainer/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-5000/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-5000/special_tokens_map.json



1번째 epoch 진행 중 ------- 5000번째 step 결과
input 문장 : [MASK]의 풍부한 실무 경험에서 나온 [MASK] 실전용 쿼리를 제공합니다
output 문장 : [저자]의 풍부한 실무 경험에서 나온 [다양한] 실용 쿼리를 제공합니다

1번째 epoch 진행 중 ------- 5100번째 step 결과
input 문장 : 전문가가 아닌 [MASK] 머 [MASK]러닝을 이해할 수 있도록 지도 학습 두명 비지도 학습에 해당하는 [MASK] [MASK]리즘을 설명 [MASK]
output 문장 : 전문가가 아닌 [사람이] 머 [##신]러닝을 이해할 수 있도록 지도 학습과 비지도 학습에 해당하는 [기초] [알고]리즘을 설명 [##한다]

1번째 epoch 진행 중 ------- 5200번째 step 결과
input 문장 : 이러한 유 [MASK] 동작 [MASK] 중앙집중형 [MASK]러를 통해 미리 프로그래밍되어 있거나 [MASK] 각 장비 [MASK] 특정한 트래픽을 [MASK]했을 때 [MASK] 동작을 [MASK]할지를 [MASK]러에 물을 수도 있다
output 문장 : 이러한 유 [##형으로] 동작 [##은] 초집중형 [컬]러를 통해 미리 프로그래밍되어 있거나 [혹은] 각 장비 [##에] 특정한 트래픽을 [사용]했을 때 [어떤] 동작을 [해야]할지를 [에]러에 물을 수도 있다

1번째 epoch 진행 중 ------- 5300번째 step 결과
input 문장 : 그 시절 엔지니어링 팀에서 년 동안 우리 팀이 일궈낸 것들은 여전히 [MASK] 커 [MASK]에서 가장 빛나는 부분 [MASK] 하나입니다
output 문장 : 그 시절 엔지니어링 팀에서 년 동안 우리 팀이 일궈낸 것들은 여전히 [이] 커 [##널]에서 가장 빛나는 부분 [중] 하나입니다

1번째 epoch 진행 중 ------- 5400번째 step 결과
input 문장 : 아마존 공인 솔루션스 아키텍트 어소시에이트 자격을 취득하기 위한 [MASK]AA묏 수험서로 



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5498, training_loss=2.4147679067690095, metrics={'train_runtime': 5998.0935, 'train_samples_per_second': 58.652, 'train_steps_per_second': 0.917, 'total_flos': 1.5261062870687616e+16, 'train_loss': 2.4147679067690095, 'epoch': 2.0})